In [1]:
import os
os.environ["OPENAI_API_KEY"] = "insert api key"
os.environ["OLLAMA_HOST"] = "http://localhost:11434"
os.environ["OLLAMA_MODEL"] = "phi3:mini"

In [2]:
!pip -q install streamlit plotly faiss-cpu sentence-transformers pypdf pandas numpy openai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.2 MB/s eta 0:00:00


In [3]:
from pathlib import Path
import itertools

DATA_DIR = Path('/content/drive/MyDrive/hackathon_data')
print("Base exists:", DATA_DIR.exists(), "->", DATA_DIR)

for p in itertools.islice(DATA_DIR.glob('*'), 3):
    print("TOP:", p)

pdfs = list(DATA_DIR.rglob('*.pdf')) + list(DATA_DIR.rglob('*.PDF'))
csvs = list(DATA_DIR.rglob('*.csv')) + list(DATA_DIR.rglob('*.CSV'))
print(f"Found PDFs: {len(pdfs)}  |  CSVs: {len(csvs)}")

print("Sample PDFs:", [p.as_posix() for p in pdfs[:5]])
print("Sample CSVs:", [p.as_posix() for p in csvs[:5]])

Base exists: True -> /content/drive/MyDrive/hackathon_data
TOP: /content/drive/MyDrive/hackathon_data/Covid
TOP: /content/drive/MyDrive/hackathon_data/Diabetes
TOP: /content/drive/MyDrive/hackathon_data/Heart_Attack
Found PDFs: 20  |  CSVs: 4
Sample PDFs: ['/content/drive/MyDrive/hackathon_data/Covid/Big_Data_Analytics_on_Irritating_Anxiety_of_Long_COVID.pdf', '/content/drive/MyDrive/hackathon_data/Covid/Extraction_of_Features_from_Lung_Image_for_the_Detection_of_Covid-19.pdf', '/content/drive/MyDrive/hackathon_data/Covid/Analysis_and_Prediction_of_COVID-19_using_Machine_Learning.pdf', '/content/drive/MyDrive/hackathon_data/Covid/COVID-19_Update_Forecast_and_Assistant_-_An_Interactive_Web_Portal_to_Provide_Real-Time_Information_and_Forecast_COVID-19_Cases_in_Bangladesh.pdf', '/content/drive/MyDrive/hackathon_data/Covid/COVID_Detection_using_Deep_Learning.pdf']
Sample CSVs: ['/content/drive/MyDrive/hackathon_data/Covid/ctg-studies_covid.csv', '/content/drive/MyDrive/hackathon_data/Diabe

In [4]:
import os, re, json
from pathlib import Path
import numpy as np, pandas as pd
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import faiss

DATA_DIR = Path('/content/drive/MyDrive/hackathon_data')
INDEX_DIR = Path('/content/indices')
INDEX_DIR.mkdir(parents=True, exist_ok=True)

TOPICS = [p.name for p in DATA_DIR.iterdir() if p.is_dir()]
EMB_NAME = "all-MiniLM-L6-v2"
EMB_DIM = 384
model = SentenceTransformer(EMB_NAME)

def chunk(text, max_chars=1000, overlap=120):
    text = re.sub(r"\s+"," ", text or "").strip()
    out=[]; i=0
    while i < len(text):
        j=min(i+max_chars,len(text))
        cut=text.rfind(". ", i, j)
        if cut==-1 or cut < i+max_chars*0.6: cut=j
        seg=text[i:cut].strip()
        if seg: out.append(seg)
        i=max(cut-overlap, cut)
    return out

def embed(texts):
    X = model.encode(texts, normalize_embeddings=True, show_progress_bar=False)
    return np.array(X, dtype='float32')

def find_files(topic_dir: Path):
    pdfs = list(topic_dir.rglob('*.pdf')) + list(topic_dir.rglob('*.PDF'))
    csvs = list(topic_dir.rglob('*.csv')) + list(topic_dir.rglob('*.CSV'))
    return pdfs, csvs

def build_index():
    metas, corpus = [], []
    csv_frames = {}

    for t in TOPICS:
        tdir = DATA_DIR / t
        if not tdir.exists():
            print(f"[WARN] missing topic {t}")
            continue
        pdfs, csvs = find_files(tdir)
        print(f"[INFO] {t}: PDFs={len(pdfs)} CSVs={len(csvs)}")

        for pdf in sorted(pdfs):
            try:
                R = PdfReader(str(pdf))
                for pidx, page in enumerate(R.pages, start=1):
                    txt = (page.extract_text() or "").strip()
                    if not txt: continue
                    for ch in chunk(txt):
                        metas.append({"topic":t,"path":str(pdf),"page":pidx,"kind":"pdf"})
                        corpus.append(ch)
            except Exception as e:
                print(f"[WARN] skip {pdf.name}: {e}")

        csv_frames.setdefault(t, {})
        for csv in sorted(csvs):
            try:
                df = pd.read_csv(csv)
            except Exception:
                try:
                    df = pd.read_csv(csv, encoding='latin-1')
                except Exception as e:
                    print(f"[WARN] skip CSV {csv.name}: {e}")
                    continue
            schema = f"CSV {Path(csv).name} | rows={len(df)} | columns={list(df.columns)}"
            metas.append({"topic":t,"path":str(csv),"page":None,"kind":"csv"})
            corpus.append(schema)
            csv_frames[t][Path(csv).name] = df

    if not corpus:
        raise RuntimeError("No data found.")

    X = embed(corpus)
    index = faiss.IndexFlatIP(EMB_DIM); index.add(X)

    faiss.write_index(index, str(INDEX_DIR/'faiss.index'))
    with open(INDEX_DIR/'meta.json','w') as f: json.dump({"metas":metas}, f)
    pd.to_pickle(csv_frames, INDEX_DIR/'csv_cache.pkl')
    pd.Series(corpus).to_pickle(INDEX_DIR/'chunks.pkl')
    print(f"[OK] Indexed {len(corpus)} chunks")

build_index()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[INFO] Covid: PDFs=5 CSVs=1
[INFO] Diabetes: PDFs=5 CSVs=1
[INFO] Heart_Attack: PDFs=5 CSVs=1
[INFO] Knee_Injury: PDFs=5 CSVs=1
[INFO] .ipynb_checkpoints: PDFs=0 CSVs=0
[OK] Indexed 874 chunks


In [5]:
%%writefile /content/rag_gradio.py
import os, re, json, faiss, numpy as np, pandas as pd
from pathlib import Path
import gradio as gr
from sentence_transformers import SentenceTransformer
from datetime import datetime
from collections import defaultdict

INDEX_DIR = Path("/content/indices")
EMB_NAME = "all-MiniLM-L6-v2"
FEEDBACK_CSV = INDEX_DIR / "feedback.csv"
FEEDBACK_STATS = INDEX_DIR / "feedback_stats.json"
SIM_THRESHOLD = 0.25
MAX_PASSAGES = 5

model = SentenceTransformer(EMB_NAME)
index = faiss.read_index(str(INDEX_DIR / "faiss.index"))
metas = json.load(open(INDEX_DIR / "meta.json"))["metas"]
csv_cache = pd.read_pickle(INDEX_DIR / "csv_cache.pkl")
chunks = pd.read_pickle(INDEX_DIR / "chunks.pkl").tolist()
TOPICS = sorted({m["topic"] for m in metas})

OPENAI_KEY = os.getenv("OPENAI_API_KEY", "").strip()
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "").strip()
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "phi3:mini").strip()

def load_feedback_stats():
    if FEEDBACK_STATS.exists():
        with open(FEEDBACK_STATS, 'r') as f:
            return json.load(f)
    return {"chunk_likes": {}, "chunk_dislikes": {}, "chunk_usage": {}}

def save_feedback_stats(stats):
    with open(FEEDBACK_STATS, 'w') as f:
        json.dump(stats, f)

feedback_stats = load_feedback_stats()

def update_feedback_stats(liked, chunk_indices):
    global feedback_stats
    for idx in chunk_indices:
        idx_str = str(idx)
        if idx_str not in feedback_stats["chunk_usage"]:
            feedback_stats["chunk_usage"][idx_str] = 0
        feedback_stats["chunk_usage"][idx_str] += 1

        if liked:
            if idx_str not in feedback_stats["chunk_likes"]:
                feedback_stats["chunk_likes"][idx_str] = 0
            feedback_stats["chunk_likes"][idx_str] += 1
        else:
            if idx_str not in feedback_stats["chunk_dislikes"]:
                feedback_stats["chunk_dislikes"][idx_str] = 0
            feedback_stats["chunk_dislikes"][idx_str] += 1

    save_feedback_stats(feedback_stats)

def calculate_feedback_score(chunk_idx):
    idx_str = str(chunk_idx)
    likes = feedback_stats["chunk_likes"].get(idx_str, 0)
    dislikes = feedback_stats["chunk_dislikes"].get(idx_str, 0)
    usage = feedback_stats["chunk_usage"].get(idx_str, 0)

    if usage == 0:
        return 0.0

    like_ratio = likes / usage if usage > 0 else 0
    feedback_boost = like_ratio * 0.15
    dislike_penalty = (dislikes / usage) * 0.1 if usage > 0 else 0

    return feedback_boost - dislike_penalty

def guess_topics(q, k=2):
    labels = [f"Topic: {t.replace('_',' ')}" for t in TOPICS]
    E = model.encode([q] + labels, normalize_embeddings=True)
    sims = (E[0:1] @ E[1:].T).flatten()
    order = np.argsort(-sims)
    return [TOPICS[i] for i in order[:min(k, len(TOPICS))]]

def retrieve_with_scores(q, whitelist=None, k=12):
    qv = model.encode([q], normalize_embeddings=True).astype("float32")
    D, I = index.search(qv, k*3)
    hits = []
    for d, i in zip(D[0], I[0]):
        m = metas[i]
        if (whitelist is None) or (m["topic"] in whitelist):
            cosine_score = float(d)
            feedback_score = calculate_feedback_score(i)
            final_score = cosine_score + feedback_score
            hits.append((final_score, cosine_score, feedback_score, m, i))

    hits.sort(key=lambda x: -x[0])
    return hits[:k]

def llm_compose(question, passages, citations):
    sys = "You are a concise medical assistant. Use ONLY the provided passages. If unsure, say you don't know. Bullet key points. Cite sources as [n]. Prefer concise Markdown (bullets, tables)."
    ctx = "\n\n".join([f"[{i+1}] {p}" for i,p in enumerate(passages)])
    prompt = f"{sys}\n\nQuestion: {question}\n\nPassages:\n{ctx}\n\nWrite a short grounded answer with citations like [1], [2]."

    used_model = "None"

    if OPENAI_KEY:
        try:
            from openai import OpenAI
            client = OpenAI(api_key=OPENAI_KEY)
            resp = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role":"user","content":prompt}],
                temperature=0.2,
            )
            used_model = "GPT-4o-mini"
            return resp.choices[0].message.content, used_model
        except Exception as e:
            error_msg = str(e).lower()
            if "quota" in error_msg or "rate_limit" in error_msg or "insufficient" in error_msg:
                print(f"[WARN] OpenAI failed: {e}. Falling back to Ollama.")
            else:
                print(f"[ERROR] OpenAI error: {e}")

    if OLLAMA_HOST:
        try:
            import requests
            r = requests.post(f"{OLLAMA_HOST}/api/generate",
                              json={"model": OLLAMA_MODEL, "prompt": prompt, "stream": False},
                              timeout=120)
            if r.ok:
                used_model = f"Ollama-{OLLAMA_MODEL}"
                return r.json().get("response",""), used_model
        except Exception as e:
            print(f"[ERROR] Ollama error: {e}")

    bullets = "\n".join([f"- {p[:220]}…" for p in passages])
    used_model = "Extractive"
    return f"**Provisional answer (no LLM).**\n\n{bullets}\n\n" + " ".join(citations), used_model

def answer_question(question: str):
    if not question.strip():
        return "Please type a question.", "", "", []

    topics = guess_topics(question, k=2)
    hits = retrieve_with_scores(question, whitelist=topics, k=12)

    if not hits:
        return "I couldn't find anything relevant in your files.", "", "", []

    top_score = hits[0][0]
    top_k = hits[:MAX_PASSAGES]
    citations = [f"[{i+1}] {Path(m['path']).name}" + (f" p.{m['page']}" if m['kind']=='pdf' and m['page'] else "")
                 for i,(final,cosine,fb,m,idx) in enumerate(top_k)]
    passages = [chunks[idx] for _,_,_,_,idx in top_k]
    chunk_indices = [idx for _,_,_,_,idx in top_k]

    if top_score < SIM_THRESHOLD:
        safe = (
            "I don't have enough evidence in your documents for a confident answer. "
            "Try rephrasing or narrowing the topic. Closest sources:\n" + "\n".join(citations)
        )
        return safe, "\n".join(citations), "Low confidence", chunk_indices

    ans, used_model = llm_compose(question, passages, citations)

    rerank_info = "\n".join([
        f"[{i+1}] Score: {final:.3f} (cosine: {cosine:.3f} + feedback: {fb:+.3f})"
        for i, (final, cosine, fb, _, _) in enumerate(top_k)
    ])

    return ans, "\n".join(citations), f"Model used: {used_model}\n\nReranking scores:\n{rerank_info}", chunk_indices

def log_feedback(liked: bool, question: str, chunk_indices):
    if not question.strip():
        return "No question to give feedback on"

    FEEDBACK_CSV.parent.mkdir(parents=True, exist_ok=True)
    row = {
        "ts": datetime.utcnow().isoformat(timespec="seconds") + "Z",
        "liked": int(liked),
        "question": question,
        "chunks": "|".join(map(str, chunk_indices)),
    }
    df = pd.DataFrame([row])
    if FEEDBACK_CSV.exists():
        df.to_csv(FEEDBACK_CSV, mode="a", header=False, index=False)
    else:
        df.to_csv(FEEDBACK_CSV, index=False)

    update_feedback_stats(liked, chunk_indices)

    if liked:
        return "Glad you liked my response! Feedback recorded."
    else:
        return "Thanks for the feedback. I'll try to improve next time."

with gr.Blocks(title="MedicAI RAG", theme=gr.themes.Soft()) as demo:
    llm_status = "GPT-4o-mini (primary) -> Ollama (fallback)" if OPENAI_KEY and OLLAMA_HOST else ("GPT-4o-mini only" if OPENAI_KEY else ("Ollama only" if OLLAMA_HOST else "No LLM"))
    gr.Markdown(f"# MedicAI\n*Hybrid LLM: {llm_status} | Cosine threshold: {SIM_THRESHOLD}*")

    with gr.Row():
        question_box = gr.Textbox(
            label="Ask a question",
            placeholder="e.g., 'List prediction methods for heart disease using ML'",
            lines=2
        )

    ask_btn = gr.Button("Ask", variant="primary")

    answer_box = gr.Markdown(label="Answer")
    citations_box = gr.Textbox(label="Citations", lines=3)
    model_status = gr.Textbox(label="Model Status", interactive=False, lines=8)

    chunk_indices_state = gr.State([])

    with gr.Row():
        thumbs_up = gr.Button("👍", scale=1)
        thumbs_down = gr.Button("👎", scale=1)

    feedback_status = gr.Textbox(label="Feedback", interactive=False)

    ask_btn.click(
        fn=answer_question,
        inputs=[question_box],
        outputs=[answer_box, citations_box, model_status, chunk_indices_state]
    )

    thumbs_up.click(
        fn=lambda q, ci: log_feedback(True, q, ci),
        inputs=[question_box, chunk_indices_state],
        outputs=[feedback_status]
    )

    thumbs_down.click(
        fn=lambda q, ci: log_feedback(False, q, ci),
        inputs=[question_box, chunk_indices_state],
        outputs=[feedback_status]
    )

if __name__ == "__main__":
    demo.launch(share=True, server_name="0.0.0.0", server_port=7860)

Writing /content/rag_gradio.py


In [ ]:
!python /content/rag_gradio.py

2025-11-07 07:28:33.947009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762500513.975040    1503 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762500513.982850    1503 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762500514.001641    1503 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762500514.001699    1503 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762500514.001704    1503 computation_placer.cc:177] computation placer alr

###✅ Project Summary

MedicAI is a Retrieval-Augmented Generation (RAG) system tailored for medical document analysis. It bridges the gap between large-scale medical literature and real-time clinical or research needs by combining semantic retrieval (FAISS + Sentence Transformers) with LLM-powered grounded generation.

###🔍 Key Features
Indexing Pipeline: Extracts, chunks, and embeds documents using sentence-transformers for semantic search.


Retrieval Pipeline: Uses FAISS for vector similarity search, topic classification for precision, and feedback-based reranking to learn from user preferences.


Generation Pipeline: Employs a hybrid LLM system (GPT-4o-mini → Ollama → Extractive summarization) ensuring reliability and cost-efficiency.


Feedback Loop: Continuous learning through user feedback to improve retrieval quality.


Deployment: Built with Gradio for instant UI and hosted via Google Colab, ensuring fast prototyping and easy sharing.


###💡 Highlights

Feedback-driven improvement of retrieval relevance


Fallback resilience against API or connectivity issues


Transparent citations for medical traceability


Scalable, domain-adaptable, and privacy-aware architecture



###Credits-
RAG Solution System Design and Architecture — Soham Jain

Deployment Solutions and Gradio Application — Sankalp Wahane

